In [2]:
import pandas as pd
from transformers import AutoTokenizer


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import sys
import os
sys.path.append('/content/gdrive/MyDrive')

In [9]:
with open('/content/gdrive/MyDrive/labeled_telegram_product_price_location.txt', 'r') as file:
    lines = file.readlines()

# Process lines as needed
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter
df = pd.DataFrame(data)

In [11]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [14]:
# data

different tokenizers from HuggingFace



In [15]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"  # Change to any approprate model from Hugging Face if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [17]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

the tokenizer link: [amseg](https://github.com/uhh-lt/amharicprocessor)

In [19]:
from amseg.amharicSegmenter import AmharicSegmenter


In [20]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [21]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [22]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [23]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [24]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")


የሞተ: O
ቆዳን: O
እንዲሁም: O
ቆሻሻን: O
ለማፅዳት: O
ተመራጭ: O
ዋጋ: B-PRICE
200: I-PRICE
ብር: I-PRICE
ውስን: O
ፍሬ: O
ነው: O
ያለው: O
አድራሻ: O
መገናኛ: B-LOC
መሰረት: I-LOC
ደፋር: I-LOC
ሞል: I-LOC
ሁለተኛ: O
ፎቅ: O


In [25]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('/content/gdrive/MyDrive/final_tokens_labels.csv', index=False)

In [ ]:

# # Display results from index 50 to 80 with formatting
# for token, label in zip(aligned_tokens[:80], aligned_labels[:80]):
#     print(f"{token:<20} {label}")

In [26]:
output_df.head()

,Token,Label
0,የሞተ,O
1,ቆዳን,O
2,እንዲሁም,O
3,ቆሻሻን,O
4,ለማፅዳት,O


In [35]:
output_df.shape

(151512, 2)

Dataset used for training the custom tokenizer [dataset link](https://huggingface.co/datasets/israel/Amharic-News-Text-classification-Dataset?row=23)

In [27]:
from datasets import load_dataset
import sentencepiece as spm


In [28]:
#Load the dataset
dataset = load_dataset("israel/Amharic-News-Text-classification-Dataset")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/150M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41186 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10297 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 41186
    })
    test: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 10297
    })
})

In [29]:
# Extract the text from the dataset
texts = dataset['train']['article']


In [30]:
# Filter out None values and prepare the data for SentencePiece
filtered_texts = [text for text in texts if text is not None]  # Remove None values
text_data = "\n".join(filtered_texts)

In [31]:
# Save to a temporary file
with open('temp_text.txt', 'w', encoding='utf-8') as f:
    f.write(text_data)

In [32]:
# Train SentencePiece model
spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=AmharicSPM --vocab_size=100000')

In [ ]:
# spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=am-word --model_type=word  --vocab_size=100000')


In [33]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [ ]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data]  # Extract labels

In [ ]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [ ]:
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []
    token_ids = []

    for word, label in zip(tokens, labels):
        tokenized_ids = tokenizer.encode(word, out_type=int)  # Get token IDs
        tokenized_words = tokenizer.decode(tokenized_ids).split()  # Decode back to words

        aligned_tokens.extend(tokenized_words)  # Add decoded words to the list
        token_ids.extend(tokenized_ids)  # Add token IDs to the list

        aligned_labels.extend([label] + ['O'] * (len(tokenized_words) - 1))

    return aligned_tokens, token_ids, aligned_labels


In [ ]:
# Tokenize and align labels
custom_tokens, custom_ids, custom_labels = tokenize_and_align_labels(tokenizer, tokens, labels)


In [ ]:
# Display results from index 50 to 80 with tokenized words, their IDs, and labels
for token, token_id, label in zip(custom_tokens[:20], custom_ids[:20], custom_labels[:20]):
    print(f"{token:<25} {token_id:<10} {label}")


3pcs                      216        B-PRODUCT
silicon                   14675      I-PRODUCT
brush                     1766       I-PRODUCT
spatulas                  230        I-PRODUCT
እስከ                       19618      O
260                       87606      O
⁇                         8556       O
c                         59828      O
ሙቀት                       52958      O
መቆቆም                      17347      O
የሚችል                      46238      O
ዋጋ                        230        I-PRICE
⁇                         59         O
550ብር                     16059      O
አድራሻ                      0          O
ቁ                         1766       O
⁇                         6229       O
1                         24606      O
ስሪ                        13608      O
ኤም                        1071       O
